# Vector-search on PyMongo documentation

In [64]:
!pip install superduperdb
!pip install sphinx
!pip install sphinx-markdown-builder

In [66]:
%%bash
rm -rf mongo-python-driver
git clone git@github.com:mongodb/mongo-python-driver.git
cd mongo-python-driver
echo 'extensions.append("sphinx_markdown_builder")' >> doc/conf.py
sphinx-apidoc -f -o source pymongo/
mkdir output
sphinx-build -a -b markdown doc output

Cloning into 'mongo-python-driver'...


Creating file source/pymongo.rst.
Creating file source/modules.rst.
Running Sphinx v7.1.2
loading intersphinx inventory from https://www.gevent.org/objects.inv...
loading intersphinx inventory from https://docs.python.org/3/objects.inv...
building [mo]: all of 0 po files
writing output... 
building [markdown]: all source files
updating environment: [new config] 82 added, 0 changed, 0 removed
reading sources... [100%] tutorial49;00mmm9;00m
looking for now-outdated files... none found
pickling environment... done
checking consistency... done
preparing documents... done
copying assets... done
writing output... [100%] tutorial49;00mmm9;00m


/Users/dodo/SuperDuperDB/superduperdb/mongo-python-driver/bson/binary.py:docstring of bson.binary.Binary.as_uuid:1: WARNING: unknown node type: <desc_returns: <reference...>>
/Users/dodo/SuperDuperDB/superduperdb/mongo-python-driver/bson/codec_options.py:docstring of bson.codec_options.CodecOptions.with_options:1: WARNING: unknown node type: <desc_returns: <reference...>>
/Users/dodo/SuperDuperDB/superduperdb/mongo-python-driver/bson/datetime_ms.py:docstring of bson.datetime_ms.DatetimeMS.as_datetime:1: WARNING: unknown node type: <desc_returns: <reference...>>
/Users/dodo/SuperDuperDB/superduperdb/mongo-python-driver/bson/dbref.py:docstring of bson.dbref.DBRef.as_doc:1: WARNING: unknown node type: <desc_returns: <reference...><desc_sig_punctuation...><reference...><de ...>
/Users/dodo/SuperDuperDB/superduperdb/mongo-python-driver/bson/decimal128.py:docstring of bson.decimal128.Decimal128.from_bid:1: WARNING: unknown node type: <desc_returns: <reference...>>
/Users/dodo/SuperDuperDB/su

build succeeded, 35 warnings.

The markdown files are in output.


In [73]:
import os
import re

PARENT_DIR = './mongo-python-driver/output/api/pymongo'

documents = os.listdir(f'{PARENT_DIR}/')
store = []

for file in documents:
    with open(f'{PARENT_DIR}/{file}') as f:
        content = f.read()
    split = re.split(r'^(#{1,4}) ', content, flags=re.MULTILINE)
    split = [(split[2 * i - 1], split[2 * i]) for i in range(1, len(split) // 2)]
    last_key = None
    for item in split:
        type_ = item[0]
        content = item[1]
        key = content.split('\n')[0]
        value = '\n'.join(content.split('\n')[1:])
        info = {}
        if type_ in {'###', '####'}:
            if type_ == '###':
                if last_key is None:
                    last_key = key
                info['key'] = key
                info['parent'] = None
                last_key = key
            elif type_ == '####':
                info['parent'] = last_key
                info['key'] = key
            info['value'] = value
            info['document'] = file
            store.append(info)

mongo_client.md
change_stream.md
collection.md
write_concern.md
operations.md
server_api.md
command_cursor.md
client_options.md
uri_parser.md
read_concern.md
cursor.md
client_session.md
read_preferences.md
errors.md
pool.md
encryption.md
encryption_options.md
results.md
index.md
topology_description.md
collation.md
driver_info.md
monitoring.md
event_loggers.md
server_description.md
database.md


In [75]:
import json
with open('pymongo.json', 'w') as f:
    json.dump(store, f)

In [77]:
!curl -O https://superduperdb-public.s3.eu-west-1.amazonaws.com/pymongo.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  712k  100  712k    0     0  1460k      0 --:--:-- --:--:-- --:--:-- 1471k


In [80]:
from superduperdb.container.vector_index import VectorIndex
from superduperdb.container.listener import Listener
from superduperdb.ext.openai.model import OpenAIEmbedding
from superduperdb.db.mongodb.query import Collection

vector_index = VectorIndex(
    Listener(
        model=OpenAIEmbedding(model='ada...'),
    )
)

Init signature:
OpenAIEmbedding(
    model: str,
    identifier: str = '',
    version: Union[int, NoneType] = None,
    takes_context: bool = False,
    encoder: Union[superduperdb.container.encoder.Encoder, str, NoneType] = None,
    shape: Union[Sequence[int], NoneType] = None,
) -> None
Docstring:     
OpenAI embedding predictor

:param shape: The shape as ``tuple`` of the embedding.
File:           ~/SuperDuperDB/superduperdb/superduperdb/ext/openai/model.py
Type:           type
Subclasses:     